In [8]:
import os
import numpy as np
from scipy.interpolate import interp1d


def calculate_delta_f(phi, free):    
    # Filter data based on conditions
    A = free[phi < 0]
    B = free[(phi > 0) & (phi < 2.2)]
    
    # Calculate free energies
    fesA = -2.49 * np.logaddexp.reduce(-1 / 2.49 * A)
    fesB = -2.49 * np.logaddexp.reduce(-1 / 2.49 * B)
    return fesB - fesA

def load_data(fes_dir, seed, method):
    file = os.path.join(fes_dir, str(seed), "fes/100.dat")

    # Load data from the file
    data = np.loadtxt(file, comments='#')
    
    with open(file, 'r') as file:
        first_line = file.readline().strip()
        
    keys = first_line.split()[2:]
    phi_idx = keys.index(cv_name[method])
    free_idx = keys.index('file.free')

    phi = data[:, phi_idx]
    free = data[:, free_idx]
    return phi, free

def load_interp(fes_dir, seed, method):
    fes_file = os.path.join(fes_dir, str(seed), "fes/100.dat")
    cv_file = os.path.join(fes_dir, str(seed), "COLVAR")

    # Load data from the file
    data = np.loadtxt(fes_file, comments='#')
    
    with open(fes_file, 'r') as file:
        first_line = file.readline().strip()
        
    keys = first_line.split()[2:]
    cv_idx = keys.index(cv_name[method])
    free_idx = keys.index('file.free')

    cv_grid = data[:, cv_idx]
    free_grid = data[:, free_idx]

    # Load data from the file
    data = np.loadtxt(cv_file, comments='#')
    
    with open(cv_file, 'r') as file:
        first_line = file.readline().strip()
        
    keys = first_line.split()[2:]
    cv_idx = keys.index(cv_name[method])
    phi_idx = keys.index('phi')

    cv = data[:, cv_idx]
    phi = data[:, phi_idx]

    fes_interp = interp1d(
        cv_grid, 
        free_grid, 
        kind='linear', 
        fill_value="extrapolate"
    )

    free = fes_interp(cv)
    return phi, free

def delta_f(fes_dir, method):
    delta_fs = []
    # Iterate through all subdirectories
    for seed in range(3):
        if method in ['phi','ref']:
            phi, free = load_data(fes_dir, seed, method)
        else:
            phi, free = load_interp(fes_dir, seed, method)
            
        delta_f = calculate_delta_f(phi, free)
        if np.any(np.isinf(delta_f)):
            continue
        delta_fs.append(delta_f)

    # Calculate mean and standard deviation of delta F
    delta_f_mean = np.nanmean(delta_fs)
    delta_f_std = np.nanstd(delta_fs)
    return delta_f_mean, delta_f_std
        
ns = '10'

cv_name = {
    'ref': 'phi',
    'phi': 'phi',
    'AE': 'deep.node-0',
    'TAE': 'deep.node-0',
    'VDE': 'deep.node-0',
    'DeepTDA': 'deep.node-0',
    'DeepLDA': 'deep.node-0',
    'DeepTICA': 'deep.node-0',
    'TBG': 'deep.node-0',
}

# methods = ['ref', 'phi', 'AE', 'TAE', 'VDE', 'DeepTDA', 'DeepLDA', 'DeepTICA']
methods = ['DeepLDA', 'DeepTDA', 'TBG']
dates = ['0421154805', '0422051238', '0422033357']
means = []
stds = []
for idx, method in enumerate(methods):
	base_dir = f'/home/shpark/prj-mlcv/lib/enhance/simulations/aldp/{method}/{ns}/log'
    # date = sorted(os.listdir(base_dir))[-1]
	date = dates[idx]
	fes_dir = os.path.join(base_dir, date)
	mean, std = delta_f(fes_dir, method)
	means.append(mean)
	stds.append(std)

for method, mean, std in zip(methods, means, stds):
    print(f"{method}: {mean:.2f} +- {std:.2f}")

output = f"{ns} & " + " & ".join([f"{mean:.2f} $\\pm$ {std:.2f}" for mean, std in zip(means, stds)]) + " \\\\"
print(output)


DeepLDA: 19.98 +- 12.70
DeepTDA: 14.63 +- 2.66
TBG: 4.99 +- 1.85
10 & 19.98 $\pm$ 12.70 & 14.63 $\pm$ 2.66 & 4.99 $\pm$ 1.85 \\
